In [2]:
# ============================================
# 1) Parâmetros de conexão SQL Server (RM)
# ============================================
jdbc_hostname = "142.0.64.50"
jdbc_port = 38000
jdbc_database = "C8VSAQ_144754_RM_PD"

jdbc_username = "CLT144754LEITURA"
jdbc_password = "vnolx18963NLADF?@"   # Ideal: usar credencial/Key Vault

# JDBC CORRIGIDO -> porta com ":" (não usar vírgula)
jdbc_url = (
    f"jdbc:sqlserver://{jdbc_hostname}:{jdbc_port};"
    f"database={jdbc_database};"
    "encrypt=false;"
    "trustServerCertificate=true;"
)

print("JDBC URL:", jdbc_url)

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/lk_ponto.Lakehouse/Tables/tab_gold_fato_obra_ponto"


# ============================================
# 2) Consulta T-SQL (sua query completa)
# ============================================
consulta = """
WITH CTE_Hist_Equipe AS (
    SELECT DISTINCT
        HSE.CODCOLIGADA,
        HSE.CHAPA,
        HSE.CODEQUIPE AS EQUIPE,
        PE.DESCRICAO AS DESCRICAO_EQUIPE,
        HSE.DTMUDANCA,
        ISNULL(
            LEAD(HSE.DTMUDANCA, 1) OVER (
                PARTITION BY HSE.CODCOLIGADA, HSE.CHAPA
                ORDER BY HSE.DTMUDANCA
            ),
            CAST(GETDATE() AS DATE)
        ) AS PROXIMA_DATAMUDANCA_EQP
    FROM zmd_hst_equipe AS HSE
    LEFT JOIN pequipe AS PE
        ON HSE.CODEQUIPE = PE.CODCLIENTE
)
SELECT
    PFUNC.CODCOLIGADA                                   AS COLIGADA,
    PFUNC.NOME                                          AS NOME,
    PFUNC.CHAPA                                         AS CHAPA,
    EQP.EQUIPE,
    EQP.DESCRICAO_EQUIPE,
    PSECAO.CODIGO                                       AS CODIGO_SECAO,
    --PSECAO.DESCRICAO                                  AS SECAO,
    PFUNC.CODFUNCAO                                     AS CODIGO_FUNCAO,
    PFUNCAO.NOME                                        AS DESCRICAO_FUNCAO,
    ABATFUN.DATA                                        AS DATA,
    YEAR(ABATFUN.DATA)                                  AS ANO,
    MONTH(ABATFUN.DATA)                                 AS MES,
    DAY(ABATFUN.DATA)                                   AS DIA,
    DATAREFERENCIA                                      AS DATA_REFERENCIA,
    ABATFUN.BATIDA                                      AS RM_BATIDA,
    SUBSTRING(PFUNC.CODSECAO, 11, 4)                    AS OBRA,
    RIGHT('00' + CAST((ABATFUN.BATIDA / 60) AS varchar(2)), 2)
        + ':' +
    RIGHT('00' + CAST((ABATFUN.BATIDA - (ABATFUN.BATIDA / 60 * 60)) AS varchar(2)), 2)
                                                        AS BATIDA,
    ABATFUN.STATUS                                      AS STATUS_BATIDA,
    ABATFUN.NATUREZA                                    AS NATUREZA,
    AAFDT.JUSTIFICATIVA                                 AS JUSTIFICATIVA,
    AHORARIO.DESCRICAO                                  AS GRADE_HORARIO
FROM ABATFUN
INNER JOIN AAFDT
    ON AAFDT.CODCOLIGADA = ABATFUN.CODCOLIGADA
   AND AAFDT.ID = ABATFUN.IDAAFDT
INNER JOIN PFUNC
    ON ABATFUN.CHAPA = PFUNC.CHAPA
INNER JOIN AHORARIO
    ON PFUNC.CODCOLIGADA = AHORARIO.CODCOLIGADA
   AND PFUNC.CODHORARIO = AHORARIO.CODIGO
INNER JOIN PFUNCAO
    ON PFUNC.CODCOLIGADA = PFUNCAO.CODCOLIGADA
   AND PFUNC.CODFUNCAO = PFUNCAO.CODIGO    
INNER JOIN PSECAO
    ON PFUNC.CODCOLIGADA = PSECAO.CODCOLIGADA
   AND PFUNC.CODSECAO = PSECAO.CODIGO
LEFT JOIN CTE_Hist_Equipe AS EQP
    ON PFUNC.CODCOLIGADA = EQP.CODCOLIGADA
   AND PFUNC.CHAPA = EQP.CHAPA
   AND CAST(ABATFUN.DATA AS DATE) >= CAST(EQP.DTMUDANCA AS DATE)
   AND CAST(ABATFUN.DATA AS DATE) <  CAST(EQP.PROXIMA_DATAMUDANCA_EQP AS DATE)
WHERE ABATFUN.DATA >= DATEADD(DAY, -35, CAST(GETDATE() AS DATE))
"""


# ============================================
# 3) Ler do SQL Server (via JDBC) para Spark
# ============================================
df_ponto = (
    spark.read
    .format("jdbc")
    .option("url", jdbc_url)
    .option("user", jdbc_username)
    .option("password", jdbc_password)
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
    .option("query", consulta)
    .load()
)

# Opcional: inspecionar esquema e algumas linhas
df_ponto.printSchema()
display(df_ponto.limit(20))


# Salva no Lakehouse de destino usando caminho físico
df_resultado.write.format("delta").mode("overwrite").save(path_destino)


StatementMeta(, 9922e368-d36a-450c-98d7-1972c49f96cd, 4, Finished, Available, Finished)

JDBC URL: jdbc:sqlserver://142.0.64.50:38000;database=C8VSAQ_144754_RM_PD;encrypt=false;trustServerCertificate=true;


Py4JJavaError: An error occurred while calling o4659.load.
: com.microsoft.sqlserver.jdbc.SQLServerException: The TCP/IP connection to the host 142.0.64.50, port 38000 has failed. Error: "connect timed out. Verify the connection properties. Make sure that an instance of SQL Server is running on the host and accepting TCP/IP connections at the port. Make sure that TCP connections to the port are not blocked by a firewall.".
	at com.microsoft.sqlserver.jdbc.SQLServerException.makeFromDriverError(SQLServerException.java:231)
	at com.microsoft.sqlserver.jdbc.SQLServerException.convertConnectExceptionToSQLServerException(SQLServerException.java:282)
	at com.microsoft.sqlserver.jdbc.SocketFinder.findSocket(IOBuffer.java:2578)
	at com.microsoft.sqlserver.jdbc.TDSChannel.open(IOBuffer.java:719)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.connectHelper(SQLServerConnection.java:3523)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.login(SQLServerConnection.java:3172)
	at com.micros